# Traditional Augmentation
- In this notebook, we will be applying a data-centric approach on our baseline model. We will be using Traditional Augmentation techniques, such as Center Crop, Rotate, Color Shift, Flip, etc in this kernel.
- We will be applying these techniques in 3 different ways. Firstly, we will apply these techniques on 25% of the samples from the training dataset irrespective of their classes, and then observe the accuracy of the baseline model after training it on the augmented dataset.
- Secondly, we will be using these techniques for class balancing, and then, will train the baseline model on the balanced dataset.
- Thirdly, we will be analysing the class-wise performance of the baseline model, and will apply augmentation on only those classes on which the baseline model is not performing well.

# 1. Importing the Packages & Boilerplate Code

In [ ]:
import os
import sys
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from shutil import copyfile
from tabulate import tabulate
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix

# https://www.kaggle.com/c/ventilator-pressure-prediction/discussion/274717
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
import tensorflow.keras.layers as tfl

In [ ]:
# Setting the seeds
SEED = 0
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
# Making sure that Tensorflow is able to detect the GPU
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

In [ ]:
# These are the usual ipython objects
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Defining a function to list the memory consumed
# Only outputs variables taking at least 1MB space
def list_storage(inp_dir):
    # Get a sorted list of the objects and their sizes
    vars_defined = [x for x in inp_dir if not x.startswith('_') and x not in sys.modules and x not in ipython_vars]
    sto = sorted([(x, sys.getsizeof(globals().get(x))) for x in vars_defined], key=lambda x: x[1], reverse=True)
    sto = [(x[0], str(round((x[1] / 2**20), 2)) + ' MB') for x in sto if x[1] >= 2**20]
    print(tabulate(sto, headers = ['Variable', 'Storage (in MB)']))

# In order to use this function, use the below line of code
# list_storage(dir())

# 2. Importing the Train/Test Sets

In [ ]:
# Importing the Labelled Dataset
df_train = pd.read_csv("../input/cifar10/train_lab_x.csv")
y_train = pd.read_csv("../input/cifar10/train_lab_y.csv")
df_train = np.array(df_train)
y_train = np.array(y_train)
print(df_train.shape, y_train.shape)

# Reshaping the dataset
df_train = np.reshape(df_train, (-1, 3, 32, 32))
print(df_train.shape)

# Basic Pre-processing
# Creating a random permutation
perm = np.random.permutation(df_train.shape[0])

# Shuffling the training dataset
df_train = df_train[perm, : , : , : ]
y_train = y_train[perm]

# Reshaping, rescaling and one-hot encoding
df_train = np.transpose(np.array(df_train), (0, 2, 3, 1))
df_train = df_train / 255
y_train_oh = tf.one_hot(np.ravel(y_train), depth = 10)
print(df_train.shape, y_train_oh.shape)

# Importing the Test Dataset
df_test = pd.read_csv("../input/cifar10/test_x.csv")
y_test = pd.read_csv("../input/cifar10/test_y.csv")
df_test = np.array(df_test)
y_test = np.array(y_test)
print(df_test.shape, y_test.shape)

# Reshaping the dataset
df_test = np.reshape(df_test, (-1, 3, 32, 32))
print(df_test.shape)

# Reshaping, rescaling and one-hot encoding
df_test = np.transpose(np.array(df_test), (0, 2, 3, 1))
df_test = df_test / 255
y_test_oh = tf.one_hot(np.ravel(y_test), depth = 10)
print(df_test.shape, y_test_oh.shape)

# 3. Defining the Tensorflow Augmentations

In [ ]:
data_augmentation = tf.keras.Sequential([
    tfl.RandomFlip("horizontal_and_vertical"),
    tfl.RandomRotation(0.2),
])

# 4. Image Augmentation on 25% of the Training Dataset
## 4.1. Augmenting the Training Dataset

In [ ]:
# Visualizing a single image
ind = 16
example = df_train[ind, : , : , : ]
print(example.shape)

# Add the image to a batch.
image = tf.cast(tf.expand_dims(example, 0), tf.float32)
print(image.shape)

plt.figure(figsize=(5, 5))
for i in range(9):
    aug_image = data_augmentation(image)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(aug_image[0])
    plt.axis("off")

In [ ]:
# Creating an empty list
df_train_aug, y_train_aug = [], []

# Iterating over all the images in the dataset
for ind in tqdm(range(df_train.shape[0])):
    if np.random.uniform() <= 0.25:
        aug_image = data_augmentation(df_train[ind, : , : , : ])
        df_train_aug.append(aug_image)
        y_train_aug.append(y_train[ind])

# Sanity Checks and Transformations
df_train_aug = np.array(df_train_aug)
y_train_aug = np.reshape(np.array(y_train_aug), (-1, 1))
y_train_aug_oh = tf.one_hot(np.ravel(y_train_aug), depth = 10)
print(df_train_aug.shape, y_train_aug.shape, y_train_aug_oh.shape)

df_aug = np.concatenate([df_train, df_train_aug], axis = 0)
y_aug = np.concatenate([y_train, y_train_aug], axis = 0)
y_aug_oh = np.concatenate([y_train_oh, y_train_aug_oh], axis = 0)
print(df_aug.shape, y_aug.shape, y_aug_oh.shape)

# Creating a random permutation
perm_aug = np.random.permutation(df_aug.shape[0])

# Shuffling the augmented dataset
df_aug = df_aug[perm_aug, : , : , : ]
y_aug = y_aug[perm_aug]
y_aug_oh = y_aug_oh[perm_aug, : ]
print(df_aug.shape, y_aug.shape, y_aug_oh.shape)

## 4.2. Training the Baseline Model on the Augmented Dataset

In [ ]:
# Importing the Baseline Model Architecture
copyfile(src = "../input/dcai-rw/baseline_arch.py", dst = "../working/baseline_arch.py")
from baseline_arch import cnn_model

conv_model = cnn_model((32, 32, 3))
conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

# Creating Batches from the Augmented Dataset
aug_dataset = tf.data.Dataset.from_tensor_slices((df_train, y_train_oh)).batch(32)
history = conv_model.fit(aug_dataset, epochs = 25)

# Saving the model along with it's weights
conv_model.save('baseline_augmented_all.h5')

## 4.3. Predicting the Performance

In [ ]:
# Predicting on the Train/Test Datasets
preds_train = conv_model.predict(df_train)
preds_test = conv_model.predict(df_test)

# Finding the Predicted Classes
cls_train = np.argmax(preds_train, axis = 1)
cls_test = np.argmax(preds_test, axis = 1)

# Finding the Train/Test set Loss
print("Log-loss for Train Dataset = ", log_loss(y_train_oh, preds_train))
print("Log-loss for Test Dataset = ", log_loss(y_test_oh, preds_test))
print("Accuracy for Train Dataset = ", accuracy_score(y_train, cls_train))
print("Accuracy for Test Dataset = ", accuracy_score(y_test, cls_test))

- From the above analysis, we can observe that Traditional Augmentation on 25% of the Training Dataset doesn't help at all. Instead, if we compare the scores with those of the baseline model, we will find them lagging a bit. 
- Let's try to balance the dataset with the help of Traditional Augmentation Techniques, and see if it gets us any better results.

# 5. Image Augmentation for Class Balancing
## 5.1. Finding out the Class Imbalance

In [ ]:
y_train_reshape = np.reshape(y_train, (-1))
num_examples = np.zeros((10,))

for i in y_train_reshape:
    num_examples[i] += 1

# Number of examples from each class
num_exa = num_examples.astype('int32')

# Finding out the maximum number of examples for any class
max_exa = max(num_exa)

# Number of examples that needs to be added to each of the classes
aug_exa = [max_exa - num_exa[i] for i in range(10)]

# Creating a list of lists for storing the indices of data-points in the training dataset, class-wise
classes_ind = []
for i in range(10):
    classes_ind.append([])

for ind, clss in enumerate(y_train_reshape):
    classes_ind[clss].append(ind)

# # Transforming list of lists into numpy array
# classes_ind = np.array([np.array(xi) for xi in classes_ind])

print(num_exa)
print(aug_exa, sum(aug_exa))
print(len(classes_ind), len(classes_ind[0]))

# Creating a list for indices of images and their labels on which augmentation needs to be done
# These are randomly chosen from each class
aug_ind = []
y_train_aug = []

for i in range(10):
    indices = random.choices(classes_ind[i], k = aug_exa[i])
    aug_ind.extend(indices)
    y_train_aug.extend([i]*aug_exa[i])

print(len(aug_ind), len(y_train_aug))

## 5.2. Augmenting based on the Class Imbalance

In [ ]:
# Creating an empty list
df_train_aug = []

for ind in tqdm(aug_ind):
    aug_image = data_augmentation(df_train[ind, : , : , : ])
    df_train_aug.append(aug_image)
    
# Sanity Checks and Transformations
df_train_aug = np.array(df_train_aug)
y_train_aug = np.reshape(np.array(y_train_aug), (-1, 1))
y_train_aug_oh = tf.one_hot(np.ravel(y_train_aug), depth = 10)
print(df_train_aug.shape, y_train_aug.shape, y_train_aug_oh.shape)

df_aug = np.concatenate([df_train, df_train_aug], axis = 0)
y_aug = np.concatenate([y_train, y_train_aug], axis = 0)
y_aug_oh = np.concatenate([y_train_oh, y_train_aug_oh], axis = 0)
print(df_aug.shape, y_aug.shape, y_aug_oh.shape)

# Creating a random permutation
perm_aug = np.random.permutation(df_aug.shape[0])

# Shuffling the augmented dataset
df_aug = df_aug[perm_aug, : , : , : ]
y_aug = y_aug[perm_aug]
y_aug_oh = y_aug_oh[perm_aug, : ]
print(df_aug.shape, y_aug.shape, y_aug_oh.shape)

In [ ]:
# Finding out the number of examples in each class after augmentation
num_examples = np.zeros((10,))
for i in y_aug[:]:
    num_examples[i[0]] += 1
print(num_examples)

## 5.3. Training the Baseline Model on the Augmented Dataset

In [ ]:
# Importing the Baseline Model Architecture
copyfile(src = "../input/dcai-rw/baseline_arch.py", dst = "../working/baseline_arch.py")
from baseline_arch import cnn_model

conv_model = cnn_model((32, 32, 3))
conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

# Creating Batches from the Augmented Dataset
aug_dataset = tf.data.Dataset.from_tensor_slices((df_train, y_train_oh)).batch(32)
history = conv_model.fit(aug_dataset, epochs = 25)

# Saving the model along with it's weights
conv_model.save('baseline_augmented_cls_imbalance.h5')

## 5.4. Predicting the Performance

In [ ]:
# Predicting on the Train/Test Datasets
preds_train = conv_model.predict(df_train)
preds_test = conv_model.predict(df_test)

# Finding the Predicted Classes
cls_train = np.argmax(preds_train, axis = 1)
cls_test = np.argmax(preds_test, axis = 1)

# Finding the Train/Test set Loss
print("Log-loss for Train Dataset = ", log_loss(y_train_oh, preds_train))
print("Log-loss for Test Dataset = ", log_loss(y_test_oh, preds_test))
print("Accuracy for Train Dataset = ", accuracy_score(y_train, cls_train))
print("Accuracy for Test Dataset = ", accuracy_score(y_test, cls_test))

- From the above analysis, we can observe that Traditional Augmentation for Class Balancing doesn't help at all too, and if we compare the scores with those of the baseline model, we will find them lagging a bit. 
- Now, let's try to analyze the performance of the original baseline model on the non-augmented training set and test set, and depending on it's class-wise performance, we will perform augmentation on selected classes only. 

# 6. Image Augmentation based on Class-wise Performance
## 6.1. Finding out the Class-wise Performance of the Baseline Model

In [ ]:
# Importing the Baseline Model
conv_model = tf.keras.models.load_model("../input/dcai-rw/baseline_model.h5")

# Predicting on the Test Datasets
preds_test = conv_model.predict(df_test)

# Finding the Predicted Classes
cls_test = np.argmax(preds_test, axis = 1)

# Finding the Test set Loss
print("Log-loss for Test Dataset = ", log_loss(y_test_oh, preds_test))
print("Accuracy for Test Dataset = ", accuracy_score(y_test, cls_test))

In [ ]:
# Creating a list of lists for storing the class-wise indices of the Test Dataset
indices = []
for _ in range(10):
    indices.append([])
for ind, y in enumerate(y_test):
    indices[y[0]].append(ind)

# Creating list of lists values
y_test_cls = []
y_test_oh_cls = []
preds_test_cls = []
cls_test_cls = []

for _ in range(10):
    y_test_cls.append([])
    y_test_oh_cls.append([])
    preds_test_cls.append([])
    cls_test_cls.append([])

for ind, y in enumerate(y_test):
    label = y[0]
    y_test_cls[label].append(y_test[ind])
    y_test_oh_cls[label].append(y_test_oh[ind])
    preds_test_cls[label].append(preds_test[ind])
    cls_test_cls[label].append(cls_test[ind])

# Sanity Checks
print(len(y_test_cls), len(y_test_oh_cls), len(preds_test_cls), len(cls_test_cls))
print(len(y_test_cls[4]), len(y_test_oh_cls[3]), len(preds_test_cls[2]), len(cls_test_cls[1]))
print(len(y_test_oh_cls[3][42]), len(y_test_oh_cls[5][49]))
print()

# Finding the Class-Wise Log-loss and Accuracy on the Test Set
for i in range(10):
    print('For Class ', i)
    print("Log-loss = ", log_loss(y_test_oh_cls[i], preds_test_cls[i]))
    print("Accuracy = ", accuracy_score(y_test_cls[i], cls_test_cls[i]))
    print()

## 6.2. Augmenting the Train Dataset

In [ ]:
# Creating a list of lists for storing the indices of data-points in the training dataset, class-wise
classes_ind = []
y_train_reshape = np.reshape(y_train, (-1))
for i in range(10):
    classes_ind.append([])
for ind, clss in enumerate(y_train_reshape):
    classes_ind[clss].append(ind)
    
# Creating a list for indices of images and their labels on which augmentation needs to be done
# These are randomly chosen from each of the underperforming classes
aug_ind = []
y_train_aug = []

# As for the under-performing classes, we will simply be choosing those classes having accuracy
# less than 80%
und_cls = [2, 3, 4, 5]

# Creating an empty list
df_train_aug, y_train_aug = [], []

# 50% Sampling
for clas in und_cls:
    for ind in tqdm(classes_ind[clas]):
        if np.random.uniform() <= 0.5:
            aug_image = data_augmentation(df_train[ind, : , : , : ])
            df_train_aug.append(aug_image)
            y_train_aug.append(y_train[ind])

# Sanity Checks and Transformations
df_train_aug = np.array(df_train_aug)
y_train_aug = np.reshape(np.array(y_train_aug), (-1, 1))
y_train_aug_oh = tf.one_hot(np.ravel(y_train_aug), depth = 10)
print(df_train_aug.shape, y_train_aug.shape, y_train_aug_oh.shape)

df_aug = np.concatenate([df_train, df_train_aug], axis = 0)
y_aug = np.concatenate([y_train, y_train_aug], axis = 0)
y_aug_oh = np.concatenate([y_train_oh, y_train_aug_oh], axis = 0)
print(df_aug.shape, y_aug.shape, y_aug_oh.shape)

# Creating a random permutation
perm_aug = np.random.permutation(df_aug.shape[0])

# Shuffling the augmented dataset
df_aug = df_aug[perm_aug, : , : , : ]
y_aug = y_aug[perm_aug]
y_aug_oh = y_aug_oh[perm_aug, : ]
print(df_aug.shape, y_aug.shape, y_aug_oh.shape)

## 6.3. Training the Baseline Model on the Augmented Dataset

In [ ]:
# Importing the Baseline Model Architecture
copyfile(src = "../input/dcai-rw/baseline_arch.py", dst = "../working/baseline_arch.py")
from baseline_arch import cnn_model

conv_model = cnn_model((32, 32, 3))
conv_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

# Creating Batches from the Augmented Dataset
aug_dataset = tf.data.Dataset.from_tensor_slices((df_train, y_train_oh)).batch(32)
history = conv_model.fit(aug_dataset, epochs = 25)

# Saving the model along with it's weights
conv_model.save('baseline_augmented_cls_performance_wise.h5')

## 6.4. Predicting the Performance

In [ ]:
# Predicting on the Train/Test Datasets
preds_train = conv_model.predict(df_train)
preds_test = conv_model.predict(df_test)

# Finding the Predicted Classes
cls_train = np.argmax(preds_train, axis = 1)
cls_test = np.argmax(preds_test, axis = 1)

# Finding the Train/Test set Loss
print("Log-loss for Train Dataset = ", log_loss(y_train_oh, preds_train))
print("Log-loss for Test Dataset = ", log_loss(y_test_oh, preds_test))
print("Accuracy for Train Dataset = ", accuracy_score(y_train, cls_train))
print("Accuracy for Test Dataset = ", accuracy_score(y_test, cls_test))

- From the above analysis, we can observe that Traditional Augmentation based on class-wise performance doesn't help at all too, and if we compare the scores with those of the baseline model, we will find them lagging a bit. 